In [92]:
import pandas as pd
import os
from sklearn import preprocessing
from collections import deque
import random
import numpy as np
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

#df = pd.read_csv("crypto_data/LTC-USD.csv", names = ["time", "low","high","open", "close", "volume"])
#print(df.head())

In [93]:
SEQ_LEN = 60
FUTURE_PERIOD_PREDICT = 3
RATIO_TO_PREDICT = "LTC-USD"
EPOCHS = 10
BATCH_SIZE = 64
NAME = f"{RATIO_TO_PREDICT}-{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"

def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0
    
def preprocess_df(df):
    df=df.drop('future',1)
    
    for col in df.columns:
        if col != "target":
            df[col] = df[col].pct_change()
            df.dropna(inplace = True)
            df[col] = preprocessing.scale(df[col].values)
    df.dropna(inplace=True)
    sequential_data = []
    prev_days = deque(maxlen=SEQ_LEN)
    '''
    print(df.head())
    for c in df.columns:
        print(c)        
    '''
    #list of lists
    for i in df.values:
        prev_days.append([n for n in i[:-1]])
        if len(prev_days) == SEQ_LEN:
            sequential_data.append([np.array(prev_days), i[-1]])
            
    random.shuffle(sequential_data)
    
    buys = []
    sells = []
    
    for seq, target in sequential_data:
        if target == 0:
            sells.append([seq,target])
        elif target == 1:
            buys.append([seq,target])
    
    random.shuffle(buys)
    random.shuffle(sells)
    
    lower = min(len(buys), len(sells))
    
    buys = buys[:lower]
    sells = sells[:lower]
        
    sequential_data = buys+sells
    
    random.shuffle(sequential_data)
    x=[]
    y=[]
    
    for seq, target in sequential_data:
        x.append(seq)
        y.append(target)
    
    return np.array(x), y


In [94]:
main_df = pd.DataFrame()
ratios = ["BTC-USD","LTC-USD","ETH-USD","BCH-USD"]

for ratio in ratios:
    dataset = f"crypto_data/{ratio}.csv"
    df = pd.read_csv(dataset, names = ["time", "low","high","open", "close", "volume"])
    #print(df.head())
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume" }, inplace=True)
    df.set_index("time", inplace = True)
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    
    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)
    
main_df['future'] = main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT)


In [95]:
main_df['target'] = list(map(classify, main_df[f"{RATIO_TO_PREDICT}_close"], main_df["future"]))
print(main_df[[f"{RATIO_TO_PREDICT}_close" ,"future","target"]].head())

            LTC-USD_close     future  target
time                                        
1528968660      96.580002  96.500000       0
1528968720      96.660004  96.389999       0
1528968780      96.570000  96.519997       0
1528968840      96.500000  96.440002       0
1528968900      96.389999  96.470001       1


In [96]:
times = sorted(main_df.index.values)
last_5pct = times[-int(0.05*len(times))]
print(last_5pct)

validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]


print(main_df)

1534922100
            BTC-USD_close  BTC-USD_volume  LTC-USD_close  LTC-USD_volume  \
time                                                                       
1528968660    6489.549805        0.587100      96.580002        9.647200   
1528968720    6487.379883        7.706374      96.660004      314.387024   
1528968780    6479.410156        3.088252      96.570000       77.129799   
1528968840    6479.410156        1.404100      96.500000        7.216067   
1528968900    6479.979980        0.753000      96.389999      524.539978   
...                   ...             ...            ...             ...   
1534921800    6686.250000        0.478039      57.509998       18.782650   
1534921860    6686.250000        0.440793      57.500000        8.449425   
1534921920    6686.250000        2.678847      57.509998        6.070000   
1534921980    6686.250000        0.220156      57.509998       15.697691   
1534922040    6685.000000        6.401611      57.509998        0.212400   



In [97]:
#preprocess_df(main_df)
train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)

In [98]:
print(f"train data: {len(train_x)} validation: {len(validation_x)}")
print(f"Dont buys: {train_y.count(0)}, buys: {train_y.count(1)}")
print(f"VALIDATION Dont buys: {validation_y.count(0)}, buys: {validation_y.count(1)}")

train data: 69188 validation: 3062
Dont buys: 34594, buys: 34594
VALIDATION Dont buys: 1531, buys: 1531


In [ ]:
model = Sequential()

model.add(LSTM(128,input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128,input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(128,input_shape=(train_x.shape[1:])))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(2, activation="softmax"))

opt = tf.keras.optimizers.Adam(lr=0.001,decay=1e-6)

model.compile(loss='sparse_categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])

tensorboard = TensorBoard(log_dir=f'logs/{NAME}')

checkpoint_filepath = "models/RNN_Final-{epoch:02d}-{val_accuracy.3f}.hd5"
checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
validation_x = np.asarray(validation_x)
validation_y = np.asarray(validation_y)

history = model.fit(
    train_x, train_y,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(validation_x, validation_y),
    callbacks=[tensorboard],
)

Epoch 1/10
1082/1082 [==============================] - 422s 386ms/step - loss: 0.7498 - accuracy: 0.5101 - val_loss: 0.7090 - val_accuracy: 0.5039
Epoch 2/10
1082/1082 [==============================] - 391s 362ms/step - loss: 0.6909 - accuracy: 0.5308 - val_loss: 0.6901 - val_accuracy: 0.5232
Epoch 3/10
1082/1082 [==============================] - 405s 374ms/step - loss: 0.6907 - accuracy: 0.5198 - val_loss: 0.7065 - val_accuracy: 0.5062
Epoch 4/10
1082/1082 [==============================] - 404s 373ms/step - loss: 0.6906 - accuracy: 0.5240 - val_loss: 0.6895 - val_accuracy: 0.5304
Epoch 5/10
1082/1082 [==============================] - 387s 358ms/step - loss: 0.6888 - accuracy: 0.5370 - val_loss: 0.6851 - val_accuracy: 0.5395
Epoch 6/10
 216/1082 [====>.........................] - ETA: 4:57 - loss: 0.6884 - accuracy: 0.5508